In [1]:
require("/home/amit/Downloads/SemVII/QuBase.jl/src/QuBase.jl")
using QuBase
require("/home/amit/Downloads/SemVII/QuDynamics.jl/src/QuDynamics.jl")
using QuDynamics
using BenchmarkLite

In [2]:
type SampleProc{Alg} <: Proc end

In [3]:
type JaynesCummings
    N::Int
    wc::Float64
    wa::Float64
    g::Float64
    kappa::Float64
    gamma::Float64
    use_rwa::Bool
    ntraj
    tlist
end

In [4]:
Base.string{Alg}(::SampleProc{Alg}) = lowercase(string(Alg))

string (generic function with 9 methods)

In [5]:
Base.isvalid{Alg}(p::SampleProc{Alg}, cfg::JaynesCummings) = true

isvalid (generic function with 23 methods)

In [6]:
function operator_set(p::SampleProc, cfg::JaynesCummings)
    idc = QuArray(eye(cfg.N))
    ida = QuArray(eye(2))
    a  = tensor(lowerop(cfg.N), ida)
    sm = tensor(idc, lowerop(2))
    if cfg.use_rwa
        H = cfg.wc * a' * a + cfg.wa * sm' * sm + cfg.g * (a' * sm + a * sm')
    else
        H = cfg.wc * a' * a + cfg.wa * sm' * sm + cfg.g * (a' + a) * (sm + sm')
    end
    c_op_list = Array(QuBase.AbstractQuMatrix, 0)
    n_th_a = 0.0
    rate_1 = cfg.kappa * (1 + n_th_a)
    if rate_1 > 0.0
        push!(c_op_list, full(sqrt(rate_1) * a))
    end
    rate = cfg.kappa * n_th_a
    if rate > 0.0
        push!(c_op_list, full(sqrt(rate) * a'))
    end
    rate = cfg.gamma
    if rate > 0.0
        push!(c_op_list, full(sqrt(rate) * sm))
    end
    psi = complex(tensor(statevec(1, FiniteBasis(cfg.N)), statevec(2, FiniteBasis(2))))
    rho = psi*psi'
    qumcwfen = QuMCWFEnsemble(complex(psi), cfg.ntraj)
    rhos = Array(typeof(rho), length(cfg.tlist)-1)
    for i=1:length(cfg.tlist)-1
        rhos[i] = complex(zeros(rho))
    end
    return full(H), c_op_list, qumcwfen, rhos
end

operator_set (generic function with 1 method)

In [7]:
Base.start(p::SampleProc, cfg::JaynesCummings) = (operator_set(p, cfg))

start (generic function with 37 methods)

In [8]:
Base.length(p::SampleProc, cfg::JaynesCummings) = cfg.N

length (generic function with 50 methods)

In [9]:
function Base.run{Alg}(p::SampleProc{Alg}, cfg::JaynesCummings, s::(QuBase.QuArray, Array, QuDynamics.QuMCWFEnsemble, Array))
    for psi0 in s[3]
        i = 1
        for (t,psi) in QuPropagator(s[1], s[2], psi0, cfg.tlist, Alg())
            s[4][i] = s[4][i] + (psi*psi')/length(s[3])/norm(psi)^2
            i = i + 1
        end
    end
end

run (generic function with 4 methods)

In [10]:
Base.done(p::SampleProc, cfg, s) = nothing

done (generic function with 40 methods)

In [11]:
procs = Proc[ SampleProc{QuMCWF}()]

1-element Array{Proc,1}:
 SampleProc{QuMCWF}()

In [12]:
cfgs = JaynesCummings[JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 200, 0.:0.1:2*pi),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 300, 0.:0.1:2*pi),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 400, 0.:0.1:2*pi),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 500, 0.:0.1:2*pi)]

4-element Array{JaynesCummings,1}:
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,200,0.0:0.1:6.2)
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,300,0.0:0.1:6.2)
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,400,0.0:0.1:6.2)
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,500,0.0:0.1:6.2)

In [ ]:
rtable = run(procs, cfgs)
show(rtable; unit=:usec)


Benchmarking qumcwf ...
